# Generating Autoencoders over Parameter Grid

Imports and Constants

**Part 1:** Parameter Selection

**Part 2:** Model Creation


Based on ELF Autoencoder training by Pentz et al.

By Justin Hart, 2025


### Imports and Logging Creation

In [1]:
%matplotlib inline
import sys
import os
sys.path.append(os.path.abspath('..'))  

from fastai import *
from fastai.vision.all import *

from Model.transforms import Binarize
from Model.TensorImageNoised import PILImageNoised, RandomErasingTransform
import Model.model as model

from fastprogress.fastprogress import force_console_behavior
from fastprogress import fastprogress

from itertools import product
import logging

fastprogress.NO_BAR = False
master_bar, progress_bar = force_console_behavior()
fastprogress.master_bar, fastprogress.progress_bar = master_bar, progress_bar

import logging

logging.basicConfig(
    filename="model_generation.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode='a'  # Append to existing log
)

logger = logging.getLogger(__name__)

## Part 1: Parameter Selection
- **resnet_version_param** is the depth of the model (18, 34, 50, 101, 152)
- **latent_space_dim_param** is the dimension of the fingerprint, 1 leads to a latent space of a 7x7 layer (1, 2, ...)
- **window_size_param** is the number of eV above and below the fermi energy (1, 2, ...)
- **training_steps_param** is the number of steps the model is trained for (1, 2, ...) 
    - this was used instead of validation score or test score due to the flucuation of them throughout training
- **train_test_split_param** is the percent of band images used in training ([0, 1))
- **line_width_param** is the width of the lines in the band structure ((0,np.inf)])

In [2]:
resnet_version_param = [18, 50]
latent_space_dim_param = [1, 2, 3]
window_size_param = [1, 2, 3]
training_steps_param = [30, 60, 90, 120]
train_test_split_param = [0, 0.1, 0.5]
line_width_param = [1]

In [3]:
IMAGE_PATH = r"C:\Users\Justin\Documents\Lehigh_REU\Machine Learning\Autoencoder Files\BandData"

## Part 2: Model Creation

Runs over parameter space, creating an autoencoder for each parameter and saving it. Training steps can be seen in model_generation.log, and the validation and information is additionally saved in a .csv file.

In [ ]:
results = []

parameters = product(resnet_version_param, latent_space_dim_param, window_size_param, 
                     training_steps_param, train_test_split_param, line_width_param)

os.makedirs('models', exist_ok=True)

for i, (resnet_version, latent_space_dim, window_size, training_steps, train_test_split, line_width) in enumerate(parameters):
    try:
        logger.info(f"Training resnet{resnet_version}, latent={latent_space_dim}, window={window_size}, steps={training_steps}, split={train_test_split}, line width={line_width}")    

        # Load the data
        dls = DataBlock(
            blocks=(ImageBlock(cls=PILImageNoised), ImageBlock),
            get_items=get_image_files,
            splitter=RandomSplitter(valid_pct=train_test_split, seed=37),
            item_tfms=[Resize((224))],
            batch_tfms=[Binarize(),RandomErasingTransform(sl=0.2)]
        ).dataloaders(f'{IMAGE_PATH}/Images/bands_{10 * line_width}width_{window_size}eV_ml', bs = 64, num_workers=os.cpu_count())

        # Training model
        loss = nn.BCEWithLogitsLoss()

        sigmoid = True
        if isinstance(loss, nn.BCEWithLogitsLoss):
            sigmoid = False

        ae = model.BuildAutoEncoder(f"resnet{resnet_version}", sigmoid=sigmoid, z_channels=latent_space_dim)
        learn = Learner(dls, ae, loss_func=loss)
        if torch.cuda.is_available():
            learn.to_fp32().to('cuda')
            logger.info(f"Using GPU: {torch.cuda.get_device_name(0)}")
        else:
            logger.warning("CUDA not available. Training on CPU.")

        learn.fit_one_cycle(training_steps)


        # Save the model
        model_name = f"resnet{resnet_version}_latent{latent_space_dim}_window{window_size}eV_steps{training_steps}_split{train_test_split * 10}_width{line_width}_.pkl"
        learn.export(f"models/{model_name}")


        validation_score = learn.validate()[0] if train_test_split > 0 else None
        train_score = learn.validate(dl=learn.dls.train)[0]


        results.append({
            "resnet_version": resnet_version,
            "latent_space_dim": latent_space_dim,
            "window_size": window_size,
            "training_steps": training_steps,
            "train_test_split": train_test_split,
            "line_width": line_width,
            "validation_score": validation_score,
            "test_score": train_score
        })

        logger.info(f"Finished: Train loss = {train_score:.4f}, Valid loss = {validation_score}")

    except Exception as e:
        logger.error(f"Failed on resnet{resnet_version}, latent={latent_space_dim}, window={window_size}, steps={training_steps}, split={train_test_split}, width={line_width}")
        logger.exception(e)  


import pandas as pd
df = pd.DataFrame(results)
df.to_csv("grid_search_results.csv", index=False)


epoch,train_loss,valid_loss,time
0,0.720455,None,00:17
1,0.707795,None,00:12


c:\Users\Justin\Documents\Lehigh_REU\.venv\Lib\site-packages\fastprogress\fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")
